<a href="https://colab.research.google.com/github/fmunin/RAG-Chatbot-PowerBI/blob/main/frm_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Chatbot for Power BI

### Installing required library (*Optional*)
---
This section may not be required after initial library install.


In [43]:
# Installing the required libraries
!pip install openai==1.66.3 tiktoken==0.9.0 pypdf==5.4.0 langchain==0.3.20 langchain-community==0.3.19 langchain-chroma==0.2.2 langchain-openai==0.3.9 chromadb==0.6 colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458

### Unzipping PowerBI documentation
---
The workbook will use a pdf file containing PowerBI documentation.
On initial setup this document may be a zip file and needs to be expanded

In [ ]:
!unzip /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot/PowerBI.zip


### Importing and setting up styling
---
This section will import python colorama library which will be used to style output.

In addition, styling constants will be defined.

In [1]:
#library used for styling
from colorama import init,Fore,Back,Style #used for styling only
#CONSTANTS
color_BLUE_on_WHITE = Back.WHITE + Fore.BLUE + Style.BRIGHT
color_MAGENTA_on_WHITE = Back.WHITE + Fore.MAGENTA + Style.BRIGHT

### Mapping Google Drive
---
The document and all data stores will be maintained in a Google Drive folder.
The system will be granted permission to this drive and the default directory for the notebook will changed to the drive.

In [2]:
from google.colab import drive
import os
drive.mount('/gdrive')
urlBase="/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot"
urlPowerBI=os.path.join(urlBase,"PowerBI_db")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
#change the working directory
%cd /gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot
print("current working directory is " + Style.BRIGHT + f"{os.getcwd()}")

/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot
current working directory is /gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot


### IMPORTING  Required Libraries

In [4]:
# Importing the standard Libraries
                         # For interacting with the operating system
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI             # Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

## Chunking Source PDF File


In [6]:
#Set up directory where PDF SOURCE files
urlpdfSource = os.path.join(urlBase,"source-docs")
print("Source PDF Documents are located @ " +  color_BLUE_on_WHITE + f"{urlpdfSource}")

Source PDF Documents are located @ /gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot/source-docs


### Creating the Chunking 'Engine'
---
In order to chunk the source documents, two classes will be used:
1. *RecursiveCharacterTextSplitter*
2. *PyPDFDirectoryloader* - which will load and split ALL documents in the given directory

In [7]:
#text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [8]:
# Chunking the source pdf
loader = PyPDFDirectoryLoader(urlpdfSource)
chunks_PowerBI = loader.load_and_split(text_splitter=text_splitter)


In [14]:
# Inspecting content of first page
print("The text splitter create "+ color_BLUE_on_WHITE+ f"{len(chunks_PowerBI)} chunks of text"+ Style.RESET_ALL + ".")
print()
print(color_MAGENTA_on_WHITE+'============ Printing the First Record =================')
chunks_PowerBI[0].page_content

The text splitter create 407 chunks of text.

============ Printing the First Record =================


'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [15]:
# initialize Chroma DB collection name
collectionName_PowerBI ="PowerBI-Documentation"

# Initialize OpenAI Embedding Model

In [16]:
#defining API key
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [17]:
# Instantiate the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='text-embedding-3-small'
)

In [18]:
# Initialize a persistent Chroma client
chromadb_client = chromadb.PersistentClient(
    path= urlPowerBI
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [19]:
# Pinging the database client to check if the connection is alive
# the heartbeat method returns the current time in nanoseconds and is generally used to check if the server is alive
print("Ensuring the connection is live by checking 'hearbeat' function .")
print("The current time returned by server is " + color_BLUE_on_WHITE + f"{chromadb_client.heartbeat()}")

Ensuring the connection is live by checking 'hearbeat' function .
The current time returned by server is 1768514135267246695


In [20]:
# Confirm database is empty
print("Initial Collection count in the database = " + color_BLUE_on_WHITE+ f"{chromadb_client.count_collections()}")

Initial Collection count in the database = 1


In [21]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
vectorstore = Chroma(
    collection_name= collectionName_PowerBI,
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory= urlPowerBI
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [23]:
# confirm collection
print("Collection count in the database = " + color_BLUE_on_WHITE+ f"{chromadb_client.count_collections()}")

Collection count in the database = 1


In [24]:
# Confirm database has been populated with the collection
print(color_MAGENTA_on_WHITE +"--------List of available collections-------")
chromadb_client.list_collections()

--------List of available collections-------


['PowerBI-Documentation']

In [25]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
i = 0 # Initialize the starting index for the chunks

while i < len(chunks_PowerBI): # Iterate while the index is less than the total number of chunks
    vectorstore.add_documents( # Add documents to the vector store in batches of 500
        documents=chunks_PowerBI[i:i+500], # Get the current batch of 500 chunks
        ids=["text_" + str(i) for i in range(i, i+500)] # Assign unique IDs to each chunk in the batch
    )

    i += 500 # Increment the index by 500 to move to the next batch
    time.sleep(30) # Pause for 30 seconds to avoid rate limiting issues with the vector store


Saving DB to Zip File

In [ ]:
#ensure you change the notebook working directory
print(f"Current working Directory is {os.getcwd()}")
!zip -r PowerBI_db.zip PowerBi_db

Current working Directory is /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot
updating: PowerBi_db/ (stored 0%)
updating: PowerBi_db/chroma.sqlite3 (deflated 97%)


# CRUD Operations in ChromaDB

## Inspecting Individual Records
---



In [27]:
#creating a Persisted vector store to access records
vectorstore_persisted = Chroma(
    collection_name= collectionName_PowerBI ,
    collection_metadata={"hnsw:space": "cosne"},
    embedding_function=embedding_model,         # Embedding model used to convert text into vectors
    client=chromadb_client,                     # Persistent Chroma client initialized earlier
    persist_directory= urlPowerBI               # directory where Chroma will persist its data
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [28]:
#Define chroma collection
PowerBI_Collection= chromadb_client.get_collection(collectionName_PowerBI)

In [30]:
# how many records are in collection??

print("The current PowerBI Chroma Collection has "+ color_BLUE_on_WHITE +f"{PowerBI_Collection.count()}" + Style.RESET_ALL+" records")

The current PowerBI Chroma Collection has 407 records


In [36]:
#Inspect first 2 records with .peek method

print(color_MAGENTA_on_WHITE +"="*75)
print (" "*10,"Reviewing the first two records in collection")
print("="*75+ Style.RESET_ALL)
print()
PowerBI_Collection.peek(2)

           Reviewing the first two records in collection



{'ids': ['text_0', 'text_1'],
 'embeddings': array([[-0.0141233 , -0.03104495,  0.0387464 , ..., -0.00077508,
          0.00032513,  0.01716083],
        [ 0.0097238 ,  0.00484162,  0.02150327, ..., -0.019948  ,
         -0.01871731,  0.01614773]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  'PUBLISHED BY \nMicrosoft Press \nA division of Microsoft Corporation \nOne Microsoft Way \nRedmond, Washington 98052-6399 \nCopyright © 2016 by Microsoft Corporation \nAll rights reserved. No part of the contents of \nthis book may be reproduced or transmitted in \nany form or by any means without the written \npermission of the publisher. \nISBN: 978-1-5093-0228-4 \nMicrosoft Press books are available through \nbooksellers and distributors worldwide. If you \nneed support related to this book, email \nMicrosoft Press Support at \nmspinput@microsoft.com. Please tell us what \nyou think of this book at http://aka.ms/tellpress. \nThis book is provided “as-

## Observations On Retrieving Individual Records

## Retrieving Records based on user query

In [38]:
# Create retriever interface
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',             # Use the default method based on semantic similarity
    search_kwargs = {'k': 5}                # Retrieve top 5 most similar chunks
)

In [39]:
#Define sample query
user_query = "What is Power BI"

In [40]:
#Perform similarity search using .invoke method
print(color_MAGENTA_on_WHITE +"="*75)
print (" "*10,"The Result record ")
print("="*75+ Style.RESET_ALL)
print()
retriever.invoke(user_query)

           The Result record 



ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(id='text_23', metadata={'author': 'Joan', 'creationdate': '2016-06-13T10:18:21-04:00', 'creator': 'Adobe Acrobat Pro 10.1.16', 'moddate': '2016-06-13T21:13:38-04:00', 'page': 23, 'page_label': '24', 'producer': 'Adobe Acrobat Pro 10.1.16', 'source': '/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot/source-docs/Introducing_Power_BI.pdf', 'title': '', 'total_pages': 407}, page_content='6 C H A P T E R  1  |  Introducing Power BI \n \nFortunately, David heard about an interesting \ntool called Power BI that Microsoft created in \n2015 that might be helpful toward creating a \ncollaborative environment in which any \nstakeholder of the budgeting process can share \nhis findings with others, working together on the \ngoal. But, at this point, the name and maybe a \nmarketing video is all that David knows about \nPower BI. \nDriven by curiosity, he navigates to \nwww.powerbi.com\n and starts down his learning \npath. Figure 1-2 depicts the welcome page of \nthe Power BI website

## Observations on Retrieving Records with User Query

# RAG Q&A System for Power BI Documentation

## Retrieval Stage

In [34]:
user_query = "How to add location in powerbi in my dashboard"
relevant_document_chunks = retriever.invoke(user_query)
print (f"The query '{user_query}' returned {len(relevant_document_chunks)} records")

The query 'How to add location in powerbi in my dashboard' returned 5 records


In [33]:
# Inspecting the first document
for document in relevant_document_chunks:
    print(document.page_content.replace("\t", " "))
    break

15 C H A P T E R  1  |  Introducing Power BI 
 
Also in Figure 1-8, notice the highlighted 
pushpin icon to the right of the question box. 
You can click this to “pin” the currently displayed 
visualization to the dashboard; this way, you can 
easily see it when you connect to Power BI. 
When you click the pushpin button, Power BI 
opens the Pin To Dashboard dialog box shown in 
Figure 1-9. 
 
Figure 1-9: Using the Pin To Dashboard dialog box, 
you can choose to pin a visualization to an existing or 
a new dashboard. 
To save the newly created bar chart to the 
dashboard, click Pin. Figure 1-10 shows how 
Power BI presents the dashboard with the 
pinned bar chart. (You need to go back to the 
dashboard to see it.)


## GENERATION Stage

In [ ]:
#create PROMPT template

In [ ]:
#create user message prompt

## Generating the Response

In [6]:
#testing formatting
%%html
<div style="background-color: blue;color:#FEFEFE">This is a div</div>


In [ ]:
#construct prompt
#LLM API Call
#parse the response

## Observations